In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
from tsmom_model import *
import PPCA_MR3 as p
import numpy as np
from datetime import datetime
from linearmodels.asset_pricing import LinearFactorModel

In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:159: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  price = clean_up_columns(data)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:161: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  df=pd.DataFrame()


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:43: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  v2 = clean_up_columns(data)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:45: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  volume=pd.DataFrame()
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:47: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=250,min_periods=100,center=False).mean()
  for x in v2.columns:
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:47: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  for x in v2.columns:
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:50: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 


# Presentation to be like "Time Varying Liquidity and Momentum Profits (2015) paper

Rank by deciles, HML (8,9,10) - (1,2,3).  Save monthly returns (equal (risk?) weighted) and if you want to annualized then resample.


Then correlation matrix 
* Overall market vol?
* Market illiquidity (median Amihud through time series)


# Amihud (2002) Annual portfolio sorts

In [5]:
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:258: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).mean()
  
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:258: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).std()
  


In [6]:
sector_portfolio_sorts={}
for s in sector_zscores.keys():
    sector_portfolio_sorts[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,5)

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:223: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  rtns = price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)].mean(axis=1)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:224: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  deciles[str(i)]=deciles[str(i)].append(rtns)


In [13]:
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')
sector_PC={}
sector_variance_explained=pd.DataFrame()

no_pc=3
for sect in d_map.keys():
    try: 
        PPCA=p.PPCA()
        PPCA.fit(np.array(sector_zscores[sect]),d=no_pc)
        sector_variance_explained[sect] = PPCA.var_exp
        pcs=pd.DataFrame()
        for i in range(0,no_pc,1):
            x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
            x.set_axis(labels=sector_zscores[sect].index,axis='index')
            pcs[i]=x
        sector_PC[sect] = pcs
    except:
        print(sect)

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:258: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).mean()
  
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:258: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).std()
  


# Expirimenting with annual portfolio sorts
* PPCA common liquidity factor per asset
* Median Amihud value
* Whole market Amihud value

However the strongest looks like the first one, but nothing is strongerly predictive.

However there is a timing mismatch because this is using annual portfolio sorts

In [61]:
sec='Commodities'
#factor=sector_PC[sec][[0]]
#factor=amihud[d_map[sec]].median(axis=1).dropna()
factor=amihud.median(axis=1).dropna()
ind=sector_portfolio_sorts[sec].shift().dropna().index
mod=LinearFactorModel(sector_portfolio_sorts[sec].shift().dropna(),factor.loc[ind])
res=mod.fit(cov_type='kernel')
print(res)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  5   R-squared:                      0.0017
No. Factors:                          1   J-statistic:                    10.268
No. Observations:                   203   P-value                         0.0361
Date:                  Mon, Mar 19 2018   Distribution:                  chi2(4)
Time:                          07:55:01                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
factor      8.591e-12  1.744e-10     0.0493     0.9607  -3.332e-10   3.504e-10

Covariance estimator:
KernelCovaria

# Expirimenting with monthly portfolio sorts
* PPCA common liquidity factor per asset
* Median Amihud value
* Whole market Amihud value

However the strongest looks like the first one, but nothing is strongerly predictive.



In [64]:
mon_amihud=read_monthly(amihud=True)

In [72]:
sec='Equities'
#factor=sector_PC[sec][[0]]
factor=amihud[d_map[sec]].median(axis=1).dropna()
#factor=amihud.median(axis=1).dropna()
ind=sector_portfolio_sorts[sec].shift().dropna().index
mod=LinearFactorModel(sector_portfolio_sorts[sec].shift().dropna(),factor.loc[ind])
res=mod.fit(cov_type='kernel')
print(res)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  5   R-squared:                      0.0524
No. Factors:                          1   J-statistic:                    10.824
No. Observations:                   211   P-value                         0.0286
Date:                  Mon, Mar 19 2018   Distribution:                  chi2(4)
Time:                          08:00:07                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
factor      -5.39e-12  5.375e-12    -1.0029     0.3159  -1.592e-11   5.144e-12

Covariance estimator:
KernelCovaria

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/linearmodels/utility.py:490: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


# FHT method

In [73]:
FHT=calculate_FHT(cleansed)
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(FHT)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:268: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:269: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:270: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  return data
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:270: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  return data
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:258: FutureWarning: pd.rolling_mean is dep

In [77]:
sector_portfolio_sorts_FHT={}
for s in sector_zscores.keys():
    sector_portfolio_sorts_FHT[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,5)

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:223: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  rtns = price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)].mean(axis=1)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:224: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  deciles[str(i)]=deciles[str(i)].append(rtns)


In [76]:
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')
sector_PC_FHT={}
sector_variance_explained=pd.DataFrame()

no_pc=3
for sect in d_map.keys():
    try: 
        PPCA=p.PPCA()
        PPCA.fit(np.array(sector_zscores[sect]),d=no_pc)
        sector_variance_explained[sect] = PPCA.var_exp
        pcs=pd.DataFrame()
        for i in range(0,no_pc,1):
            x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
            x.set_axis(labels=sector_zscores[sect].index,axis='index')
            pcs[i]=x
        sector_PC_FHT[sect] = pcs
    except:
        print(sect)

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:258: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).mean()
  
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:258: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).std()
  


# Using Anual sorts

In [81]:
sec='Fixed Income'
factor=sector_PC_FHT[sec][[0]]
#factor=amihud[d_map[sec]].median(axis=1).dropna()
#factor=FHT.median(axis=1).dropna()
ind=sector_portfolio_sorts[sec].shift().dropna().index
mod=LinearFactorModel(sector_portfolio_sorts_FHT[sec].shift().dropna(),factor.loc[ind])
res=mod.fit(cov_type='kernel')
print(res)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  5   R-squared:                      0.0016
No. Factors:                          1   J-statistic:                    15.214
No. Observations:                   300   P-value                         0.0043
Date:                  Mon, Mar 19 2018   Distribution:                  chi2(4)
Time:                          08:04:57                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
0             -1.6010     6.2357    -0.2568     0.7974     -13.823      10.621

Covariance estimator:
KernelCovaria

#Using Monthly

In [84]:
FHT_mon=read_monthly(amihud=False)

In [88]:
sec='Equities'
factor=sector_PC_FHT[sec][[0]]
#factor=amihud[d_map[sec]].median(axis=1).dropna()
#factor=FHT.median(axis=1).dropna()
ind=FHT_mon[sec].shift().dropna().index
mod=LinearFactorModel(FHT_mon[sec].shift().dropna(),factor.loc[ind])
res=mod.fit(cov_type='kernel')
print(res)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  5   R-squared:                      0.0540
No. Factors:                          1   J-statistic:                    4.3057
No. Observations:                   299   P-value                         0.3662
Date:                  Mon, Mar 19 2018   Distribution:                  chi2(4)
Time:                          08:07:23                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
0              2.0258     0.9981     2.0296     0.0424      0.0696      3.9820

Covariance estimator:
KernelCovaria